In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import socket
s= socket.socket()

sc = SparkContext.getOrCreate();
ssc = StreamingContext(sc, 15)
lines = ssc.socketTextStream("localhost", 9999)
s.connect(("localhost", 9999))
# row = lines.flatMap(lambda line: line.split(","))


In [3]:
from pyspark.sql.types import StructType,StructField,StringType,LongType,DoubleType
schema = StructType([\
    StructField("id",LongType(),True),\
    StructField("gender",LongType(),True),\
    StructField("age",DoubleType(),True),\
    StructField("hypertension",LongType(),True),\
    StructField("heart_disease",LongType(),True),\
    StructField("ever_married",LongType(),True),\
    StructField("work_type",LongType(),True),\
    StructField("Residence_type",LongType(),True),\
    StructField("avg_glucose_level",DoubleType(),True),\
    StructField("bmi",DoubleType(),True),\
    StructField("smoking_status",LongType(),True),\
    ])
# schema = StructType([StructField('Name', StringType(), True), StructField('DateTime', StringType(), True), StructField('Age', StringType(), True)])

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from __future__ import print_function
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
import math 
import pyspark.sql.functions as F
spark = SparkSession.builder.master("local[*]").appName("Source").getOrCreate()

rawData = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("./healthcare-dataset-stroke-data.csv")
rawData = rawData.drop('id')
rawData = rawData.filter(rawData['bmi'] != "N/A")
rawData = rawData.filter(rawData['gender'] != "Other")

Data = rawData.withColumn('ever_married', F.when(rawData['ever_married'] == "Yes", 1)\
                        .otherwise(0))
Data = Data.withColumn('gender', F.when(rawData['gender'] == 'Male', 1)\
                       .otherwise(0))
Data = Data.withColumn('Residence_type', F.when(rawData['Residence_type'] == 'Urban', 1)\
                       .otherwise(0))
Data = Data.withColumn('work_type', F.when(rawData['work_type'] == 'Never_worked', 0)\
                       .when(rawData['work_type'] == 'children', 1)\
                       .when(rawData['work_type'] == 'Private', 2)\
                       .when(rawData['work_type'] == 'Govt_job', 3)\
                       .when(rawData['work_type'] == 'Self-employed', 4))
Data = Data.withColumn('smoking_status', F.when(rawData['smoking_status'] == 'Unknown', 0)\
                       .when(rawData['smoking_status'] == 'never smoked', 1)\
                       .when(rawData['smoking_status'] == 'smokes', 2)\
                       .when(rawData['smoking_status'] == 'formerly smoked', 3))
                
X = Data.drop('stroke')
Y = Data.select('stroke')
stk = SMOTE(random_state=42)
X_res,y_res = stk.fit_resample(X.toPandas(),Y.toPandas())
joinDF = pd.concat([X_res, y_res], axis=1, join="inner")
balancedData = spark.createDataFrame(joinDF)

trainData, testData = balancedData.randomSplit([0.7, 0.3])

featureAssembler = VectorAssembler(
    inputCols=["age","gender","hypertension","heart_disease","work_type","Residence_type","avg_glucose_level","ever_married","smoking_status","bmi"],
    outputCol="features")
columns_to_drop = ["age","gender","hypertension","heart_disease","work_type","Residence_type","avg_glucose_level","ever_married","smoking_status","bmi"]
dt = DecisionTreeClassifier(labelCol="stroke", featuresCol="features")

assembleredTestData = featureAssembler.transform(testData)
assembleredTestData = assembleredTestData.drop(*columns_to_drop)
assembleredTrainData = featureAssembler.transform(trainData)
assembleredTrainData = assembleredTrainData.drop(*columns_to_drop)

model = dt.fit(assembleredTrainData)


In [5]:
# pairs = row.map(lambda word: (word, 1))
# wordCounts = pairs.reduceByKey(lambda x, y: x + y)

def convertToStringAndPushToSocket(rdd):    
    x = rdd.collect()
    # df = spark.createDataFrame(data = rdd, schema = schema)
    # liveData = featureAssembler.transform(df)
    # liveData = liveData.drop(*columns_to_drop)
    # predictions.select("prediction", "indexedLabel", "features").show()
    # x = model.transform(liveData).select("id", "prediction").rdd.map(lambda x: [x[0],x[1]]).collect()
    # x = model.transform(liveData)
    # x = df.select("id").collect()
    # predictMess = ";".join([",".join(map(str, item)) for item in x])
    predictMess = [",".join(map(str, item)) for item in x]
    # predictMess = len(x)
    s.send(predictMess.encode())


lines.foreachRDD(convertToStringAndPushToSocket)

In [6]:
ssc.start()            
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o22.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "G:\spark\spark-3.0.3-bin-hadoop2.7\python\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "G:\spark\spark-3.0.3-bin-hadoop2.7\python\pyspark\streaming\dstream.py", line 161, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "C:\Users\Kazekage\AppData\Local\Temp\ipykernel_5104\931983858.py", line 15, in convertToStringAndPushToSocket
    s.send(predictMess.encode())
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.apache.spark.streaming.api.python.TransformFunction.apply(PythonDStream.scala:78)
	at org.apache.spark.streaming.api.python.PythonDStream$.$anonfun$callForeachRDD$1(Pyth